In [1]:
import pandas as pd
from PIL import Image, ImageDraw, ImageColor
from multiprocessing import Pool
import sys

From basic analysis, no element with timestamp at ms level (even if ts is set at ts level)
Also, some timestamps with a lot of edits
In place.csv, ts - 1490918688000, and then for all we /1000 (since second granularity sufficient)

In [2]:
df = pd.read_csv('place.csv', index_col=0)

/Users/MJ/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.columns

Index(['ts', 'user_id', 'x_coordinate', 'y_coordinate', 'color'], dtype='object')

In [4]:
df[df['x_coordinate'] >= 1000].shape

(2021, 5)

In [5]:
df[df['y_coordinate'] >= 1000].shape

(1562, 5)

In [7]:
prevPixels = df[df['ts'] < 60845]
df['ts'] -= 60845
df = df[df['ts'] >= 0]
df = df[df['x_coordinate'] <= 999]
df = df[df['y_coordinate'] <= 999]

In [9]:
def save_image(image,ts):
    #immm = image
    image.save('frames/{0}.png'.format(ts))
    

# Hex color table
colors = ["#FFFFFF","#E4E4E4","#888888","#222222","#FFA7D1","#E50000","#E59500","#A06A42","#E5D900","#94E044","#02BE01","#00E5F0","#0083C7","#0000EA","#E04AFF","#820080"]

# Convert the colors to RGB tuples
for i in range(0,len(colors)):
    colors[i]=ImageColor.getrgb(colors[i])

#We want every 15 minutes => 900 seconds
period = 15 * 60
count = 0
imPeriodIdx = 0
    
with Pool(processes=4) as pool:
    im = Image.new("RGB",(1000,1000),"white")
    pixels = im.load()
    
    for i in range(prevPixels.shape[0]):
        row = prevPixels.iloc[i]
        x = int(row['x_coordinate'])
        y = int(row['y_coordinate'])
        pixels[x,y] = colors[int(row['color'])]
        
    pool.apply(save_image, args=(im, imPeriodIdx,))
    imPeriodIdx+=1
    
    for i in range(df.shape[0]):
        
        ts_bef = 0 if i == 0 else df.iloc[i-1]['ts']
        row = df.iloc[i]
        
        count += row['ts'] - ts_bef
        while(count >= period):
            count -= period
            pool.apply(save_image, args=(im, imPeriodIdx,)) 
            imPeriodIdx+=1
        
        x = int(row['x_coordinate'])
        y = int(row['y_coordinate'])
        pixels[x,y] = colors[int(row['color'])]
        
    pool.apply(save_image, args=(im,imPeriodIdx))